In [ ]:
# ssh -L 9094:192.168.1.54:9090 preu3@114.70.9.24 -p 443

from pynq import Overlay

custom_hw = Overlay("fft.bit")

custom_hw?

dma_config = custom_hw.axi_dma_2
dma_datain = custom_hw.axi_dma_1
dam_dataout = custom_hw.axi_dma_0

dma_datain?

dma_dataout?

import numpy as np
from pynq import allocate

inBuffer = allocate(shape=(8192,), dtype='uint32')
outBuffer = allocate(shape=(8192,), dtype='uint32')
configbuffer = allocate(shape=(1,), dtype='uint32')

fftdata = np.random.randint(50, size=(8192, ),dtype=np.uint32)
np.copyto(inBuffer, fftdata)

outBuffer

configbuffer[0] = 0b000001110000000000000000 + 13

dma_config.sendchannel.transfer(configbuffer)
dma_config.sendchannel.wait()

%%timeit
dma_datain.sendchannel.transfer(inBuffer)
dma_dataout.recvchannel.transfer(outBuffer)
dma_datain.sendchannel.wait()
dma_dataout.recvchannel.wait()

outBuffer

%%timeit

npResult = np.fft.fft(fftdata)

npResult

